# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [ ]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
from decimal import Decimal


In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# confirm that our new database was created
# List the databases in MongoDB
databases = mongo.list_database_names()

# Check if "uk_food" is in the list of databases
if "uk_food" in databases:
    print("The 'uk_food' database was successfully created.")
else:
    print("The 'uk_food' database was not found.")


In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in the uk_food database:", collections)

In [ ]:
# review a document in the establishments collection
one_document = establishments.find_one()
pprint(one_document)

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [ ]:
# Create a dictionary for the new restaurant data
new_restaurant_data = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Restaurant/Cafe/Canteen",
    "BusinessTypeID": "",
    "AddressLine1": "Penang Flavours",
    "AddressLine2": "146A Plumstead Rd",
    "AddressLine3": "London",
    "AddressLine4": "",
    "PostCode": "SE18 7DY",
    "Phone": "",
    "LocalAuthorityCode": "511",
    "LocalAuthorityName": "Greenwich",
    "LocalAuthorityWebSite": "http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress": "health@royalgreenwich.gov.uk",
    "scores": {
        "Hygiene": "",
        "Structural": "",
        "ConfidenceInManagement": ""
    },
    "SchemeType": "FHRS",
    "geocode": {
        "longitude": "0.08384000",
        "latitude": "51.49014200"
    },
    "RightToReply": "",
    "Distance": 4623.9723280747176,
    "NewRatingPending": True
}


In [ ]:
# Insert the new restaurant into the collection
result = establishments.insert_one(new_restaurant_data)



In [ ]:
# Check that the new restaurant was inserted
if result.inserted_id:
    print("The new restaurant 'Penang Flavours' has been successfully inserted into the collection.")
else:
    print("Insertion failed. Please check your data and try again.")


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
business_type_query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"_id": 0, "BusinessTypeID": 1, "BusinessType": 1}

# Find and display the result
business_type_result = establishments.find_one(business_type_query, projection)

# Check if the result was found
if business_type_result:
    print("BusinessTypeID for 'Restaurant/Cafe/Canteen':")
    pprint(business_type_result)
else:
    print("No matching record found.")


3. Update the new restaurant with the `BusinessTypeID` you found.

In [ ]:
# Update the new restaurant with the correct BusinessTypeID
# Define the correct BusinessTypeID for "Restaurant/Cafe/Canteen"
correct_business_type_id = "YourCorrectBusinessTypeIDHere"

# Define the query to find the new restaurant "Penang Flavours"
new_restaurant_query = {"BusinessName": "Penang Flavours"}

# Define the update operation to set the correct BusinessTypeID
update_operation = {
    "$set": {"BusinessTypeID": correct_business_type_id}
}

# Update the new restaurant with the correct BusinessTypeID
update_result = establishments.update_one(new_restaurant_query, update_operation)

In [ ]:
# Confirm that the new restaurant was updated
if update_result.modified_count > 0:
    print("The new restaurant 'Penang Flavours' has been updated with the correct BusinessTypeID.")
else:
    print("Update operation did not modify any documents. Please check your data and query.")

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [ ]:
# Find how many documents have LocalAuthorityName as "Dover"
dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of documents with LocalAuthorityName 'Dover':", dover_count)




In [ ]:
# Delete all documents where LocalAuthorityName is "Dover"
delete_result = establishments.delete_many({"LocalAuthorityName": "Dover"})


In [ ]:
# Check if any remaining documents include Dover
remaining_dover_count = establishments.count_documents({"LocalAuthorityName": "Dover"})
print("Number of documents with LocalAuthorityName 'Dover' after deletion:", remaining_dover_count)


In [ ]:
# Check that other documents remain with 'find_one'
remaining_document = establishments.find_one()
if remaining_document:
    print("At least one document remains in the collection after deletion.")
else:
    print("No documents remain in the collection after deletion.")


5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [ ]:
# Change the data type from String to Decimal for longitude and latitude

# Define the update operation for latitude and longitude fields
update_operation = {
    "$set": {
        "geocode.longitude": Decimal("$geocode.longitude"),
        "geocode.latitude": Decimal("$geocode.latitude")
    }
}

# Use update_many to apply the update operation
update_result = establishments.update_many({}, update_operation)

# Check the number of documents updated
updated_count = update_result.modified_count
print(f"Number of documents updated: {updated_count}")


Use `update_many` to convert `RatingValue` to integer numbers.

In [ ]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [ ]:
# Change the data type from String to Integer for RatingValue
update_operation = {
    "$set": {
        "RatingValue": {"$toInt": "$RatingValue"}
    }
}

In [ ]:
# Check that the coordinates and rating value are now numbers
update_result = establishments.update_many({"RatingValue": {"$ne": None}}, update_operation)

# Check the number of documents updated
updated_count = update_result.modified_count
print(f"Number of documents updated for RatingValue: {updated_count}")